<a href="https://colab.research.google.com/github/yakhubov/Labs-algoritms/blob/master/%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D1%8B_%D0%BF%D1%805.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from dataclasses import dataclass
from Stack_and_Queue.queue import MyQueue
import random as rd


@dataclass()
class TaskData:
    time: int = None
    type: int = None


class Task:
    def __init__(self, task_type, task_time):
        self.current_task = TaskData()
        self.current_task.time = task_time
        self.current_task.type = task_type

    def __str__(self):
        return '[' + str(self.get_type()) + ',' + str(self.get_time()) + ']'

    def get_time(self):
        return self.current_task.time

    def get_type(self):
        return self.current_task.type


class TaskGenerator:
    def __init__(self):
        self.queue1 = MyQueue()
        self.queue2 = MyQueue()

    def __str__(self):
        out = str(self.queue1) + '\n' + str(self.queue2)
        return out + '\n'

    def gen_task(self):
        task = Task(rd.randint(1, 2), rd.randint(4, 8))
        if task.get_type() == 1:
            self.queue1.push(task)
        else:
            self.queue2.push(task)

    def get_task(self):
        queue = rd.randint(1, 2)
        if queue == 1 and not self.queue1.check_empty():
            task = self.queue1.pop()
        elif queue == 2 and not self.queue2.check_empty():
            task = self.queue2.pop()
        elif queue == 1 and self.queue1.check_empty():
            task = self.queue2.pop()
        elif queue == 2 and self.queue2.check_empty():
            task = self.queue1.pop()
        else:
            task = None
        return task

    def none_task(self):
        return self.queue1.check_empty() and self.queue2.check_empty()



from Stack_and_Queue.processor import Processor
from Stack_and_Queue.task import TaskGenerator


generator = TaskGenerator()
processor = Processor()

for i in range(50):
    generator.gen_task()

while True:
    task = generator.get_task()
    if processor.idle_thread():
        if not generator.none_task():
            processor.add_task(task)
        elif not processor.wait.check_empty():
            processor.add_task(processor.wait.pop())
    processor.running()
    print('Tasks\n', generator)
    print('Processor:\n', processor)
    print('Stack:', processor.wait)
    if generator.none_task() and processor.wait.check_empty() and processor.idle_proc():
        break
In [ ]:
@dataclass()
class Thread:
    work_time: int = None
    task_type: int = None
    idle: bool = True


class Processor:
    def __init__(self):
        self.thread1 = Thread()
        self.thread2 = Thread()
        self.wait = MyStack()

    def __str__(self):
        out = '|thread|type|time|idle |\n'
        out += '{:<9}{:<5}{:<5}{:<6}'.format('  1', str(self.thread1.task_type), str(self.thread1.work_time),
                                             str(self.thread1.idle)) + '\n'
        out += '{:<9}{:<5}{:<5}{:<6}'.format('  2', str(self.thread2.task_type), str(self.thread2.work_time),
                                             str(self.thread2.idle))
        return out

    def add_task(self, task: Task):
        if task.get_type() == 1:
            if self.thread1.idle:
                self.thread1.task_type = task.get_type()
                self.thread1.work_time = task.get_time()
                self.thread1.idle = False
            elif self.thread1.task_type == 2:
                denied_task = Task(self.thread1.task_type, self.thread1.work_time)
                self.thread1.task_type = task.get_type()
                self.thread1.work_time = task.get_time()
                self.wait.push(denied_task)
            else:
                self.wait.push(task)
        elif task.get_type() == 2:
            if self.thread2.idle:
                self.thread2.task_type = task.get_type()
                self.thread2.work_time = task.get_time()
                self.thread2.idle = False
            elif self.thread1.idle:
                self.thread1.task_type = task.get_type()
                self.thread1.work_time = task.get_time()
                self.thread1.idle = False
            else:
                self.wait.push(task)

    def __run_task_t1(self):
        self.thread1.work_time -= 1
        if self.thread1.work_time <= 0:
            self.thread1.idle = True
            self.thread1.task_type = None
            self.thread1.work_time = None

    def __run_task_t2(self):
        self.thread2.work_time -= 1
        if self.thread2.work_time <= 0:
            self.thread2.idle = True
            self.thread2.task_type = None
            self.thread2.work_time = None

    def running(self):
        if not self.thread1.idle:
            self.__run_task_t1()
        else:
            self.thread1.idle = True
        if not self.thread2.idle:
            self.__run_task_t2()
        else:
            self.thread2.idle = True

    def idle_thread(self):
        return self.thread1.idle or self.thread2.idle

    def idle_proc(self):
        return self.thread1.idle and self.thread2.idle
In [ ]:
from Stack_and_Queue.processor import Processor
from Stack_and_Queue.task import TaskGenerator


generator = TaskGenerator()
processor = Processor()

for i in range(50):
    generator.gen_task()

while True:
    task = generator.get_task()
    if processor.idle_thread():
        if not generator.none_task():
            processor.add_task(task)
        elif not processor.wait.check_empty():
            processor.add_task(processor.wait.pop())
    processor.running()
    print('Tasks\n', generator)
    print('Processor:\n', processor)
    print('Stack:', processor.wait)
    if generator.none_task() and processor.wait.check_empty() and processor.idle_proc():
        break